In [3]:
# Local application imports
from utils import postgresImportFile, postgresRunSqlScript, postgresGetCredentials

import pandas as pd
from sqlalchemy import create_engine

In [6]:
db_name='weather_wizards'

sql_file = 'sql/weather_wizards_schema.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

sql_file = 'sql/hurricane_staging_schema.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

sql_file = 'sql/tornado_staging_schema.sql'
result = postgresRunSqlScript(db_name, sql_file)
print(result)

Successfully executed 16 commands from sql/weather_wizards_schema.sql.
Successfully executed 6 commands from sql/hurricane_staging_schema.sql.
Successfully executed 2 commands from sql/tornado_staging_schema.sql.


In [7]:
table_name = 'tornado_details_staging'
file_location = 'data/tornado_details.csv'
results = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"{results}")     

('Successfully imported 2500 records from data/tornado_details.csv into tornado_details_staging.', 2500)


In [8]:
table_name = 'hurricane_summary_staging'
file_location = 'data/hurricane_summary.csv'
results = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"{results}")

table_name = 'hurricane_details_staging'
file_location = 'data/hurricane_details.csv'
results = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"{results}")

file_location = 'data/hurricane_details_retired.csv'
results = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"{results}")

('Successfully imported 901 records from data/hurricane_summary.csv into hurricane_summary_staging.', 901)
('Successfully imported 5001 records from data/hurricane_details.csv into hurricane_details_staging.', 5001)
('Successfully imported 7917 records from data/hurricane_details_retired.csv into hurricane_details_staging.', 7917)


In [9]:
table_name = 'radar_station'
file_location = 'data/radar_locations.csv'
results = postgresImportFile(db_name, table_name, file_location, delimiter=',', headerline=True)  
print (f"{results}")

('Successfully imported 192 records from data/radar_locations.csv into radar_station.', 192)


In [10]:
# https://simplemaps.com/data/us-zips

db_name='weather_wizards'

engine = create_engine(postgresGetCredentials(engine_type='sqlalchemy') + '/' + db_name)

# Load the dataset
zip_data = pd.read_csv('data/uszips.csv')
# zip_data.info()
zip_data_region = (
    zip_data[['city', 'state_id', 'zip', 'population', 'lat', 'lng']]
    .assign(
        # Fill NaN in 'population' with 0 and convert to int
        population=lambda zip_data: zip_data['population'].fillna(0).astype(int),
        # Fill NaN in 'zip' with 0, convert to int, then to string and zero-fill to 5 digits
        zip=lambda zip_data: zip_data['zip'].fillna(0).astype(int).astype(str).str.zfill(5)
    )
    .rename(columns={'state_id': 'state', 'zip_code': 'zip', 'population': 'population_size', 'lat': 'latitude', 'lng': 'longitude'})
)

# zip_data_region.info()

zip_data_region.to_sql('region', con=engine, if_exists='append', index=False)

784